In [1]:
!pip install dlib -vvv

Using pip 23.3.2 from /opt/conda/lib/python3.10/site-packages/pip (python 3.10)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-build-tracker-l2hqxtx1
Initialized build tracking at /tmp/pip-build-tracker-l2hqxtx1
Created build tracker: /tmp/pip-build-tracker-l2hqxtx1
Entered build tracker: /tmp/pip-build-tracker-l2hqxtx1
Created temporary directory: /tmp/pip-install-3h2x16vb
Created temporary directory: /tmp/pip-ephem-wheel-cache-afd9z0aw
1 location(s) to search for versions of dlib:
* https://pypi.org/simple/dlib/
Fetching project page and analyzing links: https://pypi.org/simple/dlib/
Getting page https://pypi.org/simple/dlib/
Found index url https://pypi.org/simple/
Looking up "https://pypi.org/simple/dlib/" in the cache
Request header has "max_age" as 0, cache bypassed
No cache entry available
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /simple/dlib/ HTTP/1.1" 200 6365
Updating cache with response from "https://py

In [2]:
TRAINSET = '/kaggle/input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml'
TESTSET = '/kaggle/input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset/labels_ibug_300W_test.xml'

In [3]:
LANDMARKS = set(list(range(36, 48)) + list(range(48,68)))

In [4]:
import re

def dataset_pruning(inputFile, outputFile):
    rows = open(inputFile).read().strip().split("\n")
    output = open(outputFile, "w")

    PART = re.compile("part name='[0-9]+'")

    for row in rows:
        parts = re.findall(PART, row)
        if len(parts) == 0:
            output.write("{}\n".format(row))
        else:
            attr = "name='"
            i = row.find(attr)
            j = row.find("'", i + len(attr) + 1)
            name = int(row[i + len(attr):j])
            if name in LANDMARKS:
                output.write("{}\n".format(row))
    output.close()
    print(f"Labels exported to {outputFile}")
    return True


In [5]:
dataset_pruning(TRAINSET, 'updated_labels_train.xml')
dataset_pruning(TESTSET, 'updated_labels_test.xml')

Labels exported to updated_labels_train.xml
Labels exported to updated_labels_test.xml


True

In [6]:
cp -r /kaggle/input/ibug-300w-large-face-landmark-dataset/ibug_300W_large_face_landmark_dataset /kaggle/working/ibug_300W_large_face_landmark_dataset

In [7]:
cp /kaggle/working/updated_labels_test.xml /kaggle/working/ibug_300W_large_face_landmark_dataset

In [8]:
cp /kaggle/working/updated_labels_train.xml /kaggle/working/ibug_300W_large_face_landmark_dataset

In [9]:
import dlib

options = dlib.shape_predictor_training_options()
options.be_verbose=True
options.cascade_depth = 14

In [10]:
UPDATED_TRAINSET = '/kaggle/working/ibug_300W_large_face_landmark_dataset/updated_labels_train.xml'
UPDATED_TESTSET = '/kaggle/working/ibug_300W_large_face_landmark_dataset/updated_labels_test.xml'

In [11]:
dlib.train_shape_predictor(UPDATED_TRAINSET, 'model_full_mouth_jmd.dat', options)

Training with cascade depth: 14
Training with tree depth: 4
Training with 500 trees per cascade level.
Training with nu: 0.1
Training with random seed: 
Training with oversampling amount: 20
Training with oversampling translation jitter: 0
Training with landmark_relative_padding_mode: 1
Training with feature pool size: 400
Training with feature pool region padding: 0
Training with 0 threads.
Training with lambda_param: 0.1
Training with 20 split tests.
Fitting trees...

Training complete
Training complete, saved predictor to file model_full_mouth_jmd.dat


Progress: 4625/7000 (66.07%). Time remaining: 2min 59s.                ining: 4min 41s.                

In [16]:
error = dlib.test_shape_predictor(UPDATED_TRAINSET, '/kaggle/working/model_full_mouth_jmd.dat')

In [17]:
error_test = dlib.test_shape_predictor(UPDATED_TESTSET, '/kaggle/working/model_full_mouth_jmd.dat')error_test = dlib.test_shape_predictor(UPDATED_TESTSET, '/kaggle/working/model_full_mouth_jmd.dat')

In [14]:
print(error, error_test)

18.972120151233604 18.972120151233604


In [15]:
def evaluate_model_speed(predictor, imagePath, tests=10):
	# initialize the list of timings
	timings = []
	# loop over the number of speed tests to perform
	for i in range(0, tests):
		# load the input image and convert it to grayscale
		image = cv2.imread(config.IMAGE_PATH)
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		# detect faces in the grayscale frame
		detector = dlib.get_frontal_face_detector()
		rects = detector(gray, 1)
		# ensure at least one face was detected
		if len(rects) > 0:
			# time how long it takes to perform shape prediction
			# using the current shape prediction model
			start = time.time()
			shape = predictor(gray, rects[0])
			end = time.time()
		# update our timings list
		timings.append(end - start)
	# compute and return the average over the timings
	return np.average(timings)